In [1]:
import pandas as pd
import glob
# from statsmodels.tsa.api import SimpleExpSmoothing

In [2]:
cols = ['Year', 'Reporter Code', 'Reporter', 'Partner Code', 'Partner', 'Trade Value (US$)']
schema = {'Year': str, 'Reporter Code': str, 'Partner Code': str}
def dfs():
    for file in glob.glob('data/*.csv'):
        yield pd.read_csv(file, encoding='latin-1', usecols=cols, 
                          dtype=schema)
df = pd.concat(dfs())
df.dropna(inplace=True)
df.rename(columns={'Reporter Code': 'Reporter_Code', 'Partner Code': 'Partner_Code', 'Trade Value (US$)': 'Trade_Value'}, inplace=True)

In [3]:
reporters = set(df.Reporter_Code.unique())
print(len(reporters), "Source nodes are present. (Some have invalid data so dropped)")
df_ = df[df.Partner_Code.isin(reporters)]  #induced sub-graph creation
print("Filtered records from", len(df), "to", len(df_))

143 Source nodes are present. (Some have invalid data so dropped)
Filtered records from 74077 to 66316


In [4]:
# Filtered DF is essentially an induced graph now. Let's save this
df_.to_csv('filtered_df.csv', header=True)

In [5]:
countries = df_.Reporter.unique()
country_to_index = dict(zip(countries, range(len(countries))))
index_to_country = dict(map(reversed, country_to_index.items()));

In [6]:
# We can now start creating tensors
import numpy as np
# First dimension is for time (10 years)
# Second dimension is for TO
N = len(countries)
tensor = np.zeros((10, N, N))

In [7]:
for i, year in enumerate(['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']):
    # Data of that particular year
    df__ = df_[df_.Year == year]
    # For the scale of this data, I think we can just iterate over it.
    for _, row in df__.iterrows():
        country_from = row['Partner']
        country_to = row['Reporter']
        tensor[i, country_to_index[country_from], country_to_index[country_to]] = row['Trade_Value']

In [8]:
sparsity = 1.0 - np.count_nonzero(tensor) / tensor.size
print(f"{sparsity*100:.2f}% of the tensor is sparse")

67.58% of the tensor is sparse


In [13]:
tensor

array([[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 2.2651200e+05, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 3.5345950e+06],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 1.6782854e+07, 0.0000000e+00]],

       [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 1.5125220e+06, 0.0000000e+00, ...,
         0.000

In [168]:
!pip install tensorly

In [9]:
# Normalizing our tensor. For the first version, let's just calculate the log
non_zeros = (tensor!=0)
tensor[non_zeros] = np.log(tensor[non_zeros])
tensor

array([[[16.53887063,  0.        , 12.35463919, ...,  0.        ,
          4.76217393,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [11.60730794,  0.        ,  0.        , ...,  0.        ,
          4.83628191,  0.        ],
        [20.9365733 ,  0.        ,  0.        , ..., 18.05097911,
          0.        ,  0.        ],
        [13.00692559,  0.        , 11.15990099, ..., 13.50037369,
         14.15336398,  0.        ]],

       [[12.07779599,  0.        ,  9.23386157, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 8.36706773,  0.        ,  0.        , ...,  

In [10]:
from tensorly.decomposition import parafac

# Can tinker with ranks
# Mask has a special significance, we should really discuss and explore more about it
weights, factors = parafac(tensor, rank=150, normalize_factors=True, mask=non_zeros, random_state=42)


In [11]:
def get_slice(A, B, T, W):
    # Given the two matrices, get the slice
    print("Shape of A", A.shape)
    print("Shape of B", B.shape)
    print("Shape of T", T.shape)
    print("Shape of W", W.shape)
    print ("Shape of np.diag(W*T)", np.diag(W*T).shape)
    return np.matmul(np.matmul(A, np.diag(W*T)), B.T)

sl = get_slice(factors[1], factors[2], factors[0][9], weights)
sl[non_zeros[9]]

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)


array([ 5.85512649, 12.46666417, 18.52192549, ..., 14.76686959,
       15.63305322, 17.11343875])

In [16]:
sl.shape[0]

143

In [12]:
# recursive implementation of exponential smoothing
# Input: list of vectors, alpha
# Output: Next vector
def exponential_smoothing_recursive(vectors, alpha, depth=-1):
    if depth==-1:                          # -1 means the last one
        depth = len(vectors)-1
    if depth == 0:                        # For the first vector, return as is.
        return vectors[0]
    return alpha * vectors[depth] + \
            (1-alpha) * exponential_smoothing_recursive(vectors, alpha, depth-1)

exponential_smoothing_recursive([1,2,3], .5)

2.25

In [14]:
next_C = exponential_smoothing_recursive(factors[0], alpha=0.5)
forecast = get_slice(factors[1], factors[2], next_C, weights)

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)


array([ 0.32155375,  0.3036457 ,  0.47594147, -0.76559932,  0.25328463,
        0.2806135 ,  0.50715998, -0.03426854, -0.07444104, -0.0497859 ,
        0.33095308,  0.29773246, -0.17293837, -0.3994588 , -0.24057897,
       -0.31647308,  0.19444822,  0.37852546,  0.255299  , -0.32420629,
       -0.42051906,  0.39439336, -0.37478292,  0.61007152, -0.68752351,
        0.45158501, -0.48343486,  0.24029969, -0.40710143, -0.27334294,
        0.50698089, -0.28275666,  0.27159073, -0.26491024, -0.47924288,
        0.39386158,  0.39700713, -0.20323859,  0.25083129, -0.4824738 ,
        0.31811343,  0.45781293,  0.30480245,  0.17648065, -0.32171316,
        0.50475516, -0.21867099,  0.44582014,  0.27159955,  0.41540045,
       -0.03503163,  0.06221233,  0.34030015,  0.28816254,  0.27281064,
       -0.54455755, -0.39309303,  0.15849018, -0.22168919, -0.81365902,
       -0.03929048,  0.46012906, -0.42345557, -0.12600082, -0.02045431,
        0.48607068,  0.38766723,  0.44050753,  0.27379869, -0.27

In [33]:
# compare prediction
num_years = 7 # 2012 - 2018, predict on 2019

C_7_2019 = exponential_smoothing_recursive(factors[0][0:num_years], alpha = 0.5)
forecast_7_2019 = get_slice(factors[1], factors[2], C_2019, weights)

forecast_7_2019

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)


array([[13.56496173,  5.95977704,  9.45067772, ...,  7.92337284,
         9.57826687, 12.8690975 ],
       [-0.71652088, -0.3957143 ,  4.80271444, ...,  8.49268168,
         4.40794681,  3.45765197],
       [-0.15352475,  7.74678302,  1.43551031, ..., -1.14648071,
        -1.10237309,  1.97143428],
       ...,
       [ 9.64543058,  8.49627076, 11.16650981, ...,  7.01823251,
        16.07565315,  7.81237962],
       [18.86947164,  6.24123373,  2.32137891, ..., 19.10047807,
         2.18271608, 10.22697383],
       [14.5324087 ,  6.27102747, 13.78820273, ..., 15.10785511,
        18.21928322, -0.78701545]])

In [34]:
num_years = 2
C_2_2019 = exponential_smoothing_recursive(factors[0][0:num_years], alpha = 0.1)
forecast_2_2019 = get_slice(factors[1], factors[2], C_2_2019, weights)
forecast_2_2019

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)


array([[ 14.2161158 ,   4.85097181,  11.54241582, ...,   6.59800869,
          5.20494006,   4.14311879],
       [  1.15180471,  -5.64818569,  11.48218121, ...,   8.53524219,
         -0.09558665,   4.31851863],
       [ -1.18318998,   3.29845272,   1.031183  , ...,  -6.85988999,
         -8.72211682,   2.81281705],
       ...,
       [ 10.67164471,   3.10622847,   8.49902233, ...,   5.7377506 ,
          7.2254091 ,  -3.20400035],
       [ 21.44634562,  -9.17935765, -23.60739746, ...,  17.82337319,
         -0.51034787,   1.02112124],
       [ 13.11501439,  -3.78519491,  10.39623173, ...,  13.47357325,
         15.0627584 ,   3.21561584]])

In [32]:
slice_2019 = get_slice(factors[1], factors[2], factors[0][7], weights)

slice_2019

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)


array([[13.30633683,  5.5326697 ,  7.52995913, ...,  9.56075062,
        11.25729484, 10.18970417],
       [ 0.11527099, -0.24504553, -1.83423466, ...,  8.62106182,
         1.20227638, -5.44216975],
       [ 0.84301926, 10.96239706,  1.9080343 , ..., -0.53442165,
         2.36698039,  1.31886309],
       ...,
       [11.11094645,  1.99858824, 14.3433042 , ..., 10.17795111,
        17.71218834,  2.41973141],
       [19.53380403, 10.00993498, 31.41693133, ..., 19.90584848,
         6.63587039, 19.34640838],
       [15.96857207,  8.58107199, 17.09999153, ..., 14.36120922,
        18.24203112,  0.51889749]])

20449

In [39]:
from sklearn.metrics import mean_squared_error
def RMSE(actual, pred):     
    return (mean_squared_error(actual,pred))**0.5

In [40]:
# calculate rmse for predictions
RMSE(slice_2019, forecast_7_2019)

3.8617856536043393

In [41]:
RMSE(slice_2019, forecast_2_2019)

7.623025531190928

In [45]:
def alpha_experiment(num_years):
    rmse = []
    for alpha in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        C = exponential_smoothing_recursive(factors[0][0:num_years], alpha = alpha)
        forecast = get_slice(factors[1], factors[2], C, weights)
        rmse.append(RMSE(slice_2019, forecast))
        
    return rmse

In [49]:
experiments = []
for i in range(1, 8):
    experiments.append(alpha_experiment(i))
    
experiments

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag

[[7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615,
  7.9232881577398615],
 [7.623025531190928,
  7.361509722052515,
  7.14299773530943,
  6.971534409688772,
  6.850653364781381,
  6.783059423548,
  6.770348800649281,
  6.812828688701924,
  6.909481219369783,
  7.058081242102878],
 [7.298189608274014,
  6.847697055379872,
  6.557019955559121,
  6.402969476263094,
  6.35729271430437,
  6.391367291918184,
  6.480336053119316,
  6.605614648620121,
  6.755819288286121,
  6.926677947412208],
 [6.969955104764222,
  6.376148494002814,
  6.055861918772057,
  5.92090054397157,
  5.898967821489927,
  5.941492293453789,
  6.021485927021099,
  6.127214784051347,
  6.25598575423602,
  6.409794521981719],
 [6.611541086362098,
  5.904391862904516,
  5.589243103987273,
  5.488338328025537,
  5.488023197440405,
  5.529252803931952,
  5.585667984432773,
  5.64

In [43]:
num_years = 2
rmse = []
for alpha in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    C_2_2019 = exponential_smoothing_recursive(factors[0][0:num_years], alpha = alpha)
    forecast_2_2019 = get_slice(factors[1], factors[2], C_2_2019, weights)
    rmse.append(RMSE(slice_2019, forecast_2_2019))
    
rmse

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag

[7.623025531190928,
 7.361509722052515,
 7.14299773530943,
 6.971534409688772,
 6.850653364781381,
 6.783059423548,
 6.770348800649281,
 6.812828688701924,
 6.909481219369783,
 7.058081242102878]

In [44]:
num_years = 7
rmse = []
for alpha in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    C = exponential_smoothing_recursive(factors[0][0:num_years], alpha = alpha)
    forecast = get_slice(factors[1], factors[2], C, weights)
    rmse.append(RMSE(slice_2019, forecast))
    
rmse

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag

[5.780328855597892,
 4.717522033512287,
 4.213490822019018,
 3.9720804652145167,
 3.8617856536043393,
 3.8268002305384714,
 3.838049932307209,
 3.876712383369916,
 3.930305180139127,
 3.991582426186814]

In [52]:
# based on paper
# factor matrix C
# predict same col in next time slice C(K+1)
# takes in factors produced by parafac and years to forecast (default is 1) 
# returns list of forecasted vectors, last n in list is the forecasted n years

# Looks like you are obtaining the forecasts f1, f2, f3, etc using the temporal fronts t1, t2, t3
# and then smoothing the forecasts i.e. f_n+1
# The correct approach here is to smooth the temporal fronts i.e. get t_n+1 and then
# use it to generate the f_n+1.
def exponential_smoothing(factors, weights, forecast = 1):
    num_slices = len(factors[0])  # Temporal factors
    alpha = 0.5
    
    sl = get_slice(factors[1], factors[2], factors[0][0], weights)  # slice_0
    forecasts = [sl]
    
    for i in range(0, num_slices + forecast):
        # get slice
        s_t = forecasts[i]
        s_t1 = np.zeros((s_t.shape[0], s_t.shape[1]))
        
        past_c_k_r = []
        for k in range(0, s_t.shape[0]):
            
            for r in range(0, s_t.shape[1]):
                c_k_r = s_t[k][r]
                next_c_k_r = 0
                           
                for num in range(0, k):
                    smooth = alpha * ((1-alpha) ** num)
                    next_c_k_r += smooth * past_c_k_r[-num]
                
                past_c_k_r.append(next_c_k_r)
                    
                s_t1[k,r] = next_c_k_r
                
        forecasts.append(s_t1)
        
    return forecasts
         
        
        

In [53]:
# testing expo smoothing
forecasts = exponential_smoothing(factors, weights)
print(forecasts[-1])

Shape of A (143, 150)
Shape of B (143, 150)
Shape of T (150,)
Shape of W (150,)
Shape of np.diag(W*T) (150, 150)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [54]:
forecasts[2]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

array([ 6.08449941, 17.9035443 , 12.39185072, ..., 16.69321177,
        9.42843118, 16.63586833])